In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 guardrail-ml==0.0.12 tensorboard
!apt-get -qq install poppler-utils tesseract-ocr
!pip install -q unstructured["local-inference"]==0.7.4 pillow

In [ ]:
import pandas as pd
# our dataset
df =  pd.read_excel('/content/RelevanceData.xlsx' )

In [ ]:
df.tail()

,Description,Relevances
85,Check the number of processors is equal to 1,number of processors = 1
86,Give all the files of the folder,"files of folder ""/etc"""
87,Give the specific files of the folder,"files whose (name of it ends with "".conf"") of ..."
88,Give the multiple properties of files of folder,"it of files of folder ""/etc"""
89,Check the existence of the package,"if (exists packages ""libxml2"" of rpm) then ""Ye..."


In [ ]:
import os
import torch
from datasets import load_dataset, Dataset4
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Load the tokenizer
model_name = 'meta-llama/Llama-2-7b-chat-hf'  # model
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Initialize counters for successful and total tokenizations
total_samples = len(df)
successful_tokenizations = 0

# Iterate through your dataset and tokenize each text sample
for index, row in df.iterrows():
    text = row['Relevances']

    # Tokenize the text
    tokens = tokenizer.tokenize(text)

    words = text.split()

    # Calculate the number of words
    word_count = len(words)

    # Check if tokenization was successful (tokens are not empty)
    if len(tokens) >= word_count:
        successful_tokenizations += 1

# Calculate the percentage of successful tokenizations
percentage_successful = (successful_tokenizations / total_samples) * 100

print(f"Percentage of successful tokenizations: {percentage_successful:.2f}%")


Percentage of successful tokenizations: 100.00%


In [ ]:

# Used for multi-gpu
local_rank = -1 # default if working on the single GPU
per_device_train_batch_size = 4 # specifies the number of training examples that are processed simultaneously on each device
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1 # the model will accumulate gradients over four consecutive batches before updating the weights.This technique can be useful when dealing with limited GPU or TPU memory,
learning_rate = 2e-4
max_grad_norm = 0.3
weight_decay = 0.001 #  prevent overfitting by adding a penalty term to the loss function based on the magnitude of the model's weights.
lora_alpha = 16 # scaling factor
lora_dropout = 0.1 #  This parameter is used to avoid overfitting. This technique basically drop-outs some of the neurons during both forward and backward propagation
lora_r = 16 # his is the dimension of the low-rank matrix,
max_seq_length = None

# The model that you want to train from the Hugging Face hub
model_name = 'meta-llama/Llama-2-7b-chat-hf'

# Fine-tuned model name
new_model = "llama-2-7b-guanaco-dolly-mini"


# Activate 4-bit precision base model loading
use_4bit = True

# Activate nested quantization for 4-bit base models
use_nested_quant = False

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16" # float 16 and bfloat 16 both are memory efficient but f16 has higher precision and b16 is faster

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4" #

# Number of training epochs
num_train_epochs = 2

# Enable fp16 training, (bf16 to True with an A100)
fp16 = False

# Enable bf16 training
bf16 = False

# Use packing dataset creating
packing = False # This technique helps ensure that sequences of different lengths can be processed in parallel

# Enable gradient checkpointing
gradient_checkpointing = True #  selectively re-computes intermediate activations during the backward pass instead of storing them all,performing some extra computation to reduce memory usage

# Optimizer to use, original is paged_adamw_32bit
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine, and has advantage for analysis)
lr_scheduler_type = 'constant'

# Number of optimizer update steps, 10K original, 20 for demo purposes
max_steps = -1

# Fraction of steps to do a warmup for
warmup_ratio = 0.03 # learning rate scheduling strategy that gradually increases the learning rate from its initial value to its full value over a certain number of training steps or epochs.

# Group sequences into batches with same length (saves memory and speeds up training considerably)
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 10

# Log every X updates steps
logging_steps = 1

# The output directory where the model predictions and checkpoints will be written
output_dir = "./results"

# Load the entire model on the GPU 0
device_map = {"": 0}

# Visualize training
report_to = "tensorboard"

# Tensorboard logs
tb_log_dir = "./results/logs"


In [ ]:
def load_model(model_name):
    # Load tokenizer and model with QLoRA configuration
    compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=use_4bit,
        bnb_4bit_quant_type=bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=use_nested_quant,
    )

    if compute_dtype == torch.float16 and use_4bit:
        major, _ = torch.cuda.get_device_capability()
        if major >= 8:
            print("=" * 80)
            print("Your GPU supports bfloat16, you can accelerate training with the argument --bf16")
            print("=" * 80)

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map=device_map,
        quantization_config=bnb_config
    )

    model.config.use_cache = False
    model.config.pretraining_tp = 1

    # Load LoRA configuration
    peft_config = LoraConfig(
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        r=lora_r,
        bias="none",
        task_type="CAUSAL_LM",) # only left side of token available to  predict the future token

    # Load Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer, peft_config

In [ ]:
model, tokenizer, peft_config = load_model(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset = Dataset.from_pandas(df) # compatible to hugging face your dataset

In [ ]:
def format_data(sample):
    instruction = f"<s>[INST] {sample['Description']}"
    #context = f"Here's some context: {sample['context']}" if len(sample["context"]) > 0 else None
    response = f" [/INST] {sample['Relevances']}"
    # join all the parts together
    prompt = "".join([i for i in [instruction, response] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_data(sample)}{tokenizer.eos_token}"
    return sample


dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
dataset

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 90
})

In [ ]:
from accelerate import Accelerator
accelerator = Accelerator()
model, tokenizer = accelerator.prepare(model, tokenizer)

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    # evaluation_strategy="epoch",
    # save_strategy="epoch",
    # load_best_model_at_end=True,


)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,

)

trainer.train()
trainer.model.save_pretrained(output_dir)

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Step,Training Loss
1,5.419200
2,7.181500
3,6.592000
4,7.665300
5,7.077300
6,5.699600
7,5.614200
8,5.681300
9,5.333900
10,5.814800


In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Give the processor information"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100,temperature = 0.6)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Give the processor information [/INST] processor vendor name : Hewlett-Packard processor model name : 47942F processor type : CPU processor family : Intel processor model name : 47942F processor stepping : 685 flags : 0x00000000000000000000000000000000000


In [ ]:
prompt = "Give all the files of the folder"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Give all the file of the folder [/INST] file of folder "system" of folder "Windows" of folder "drivers" of folder "HKEY_LOCAL_MACHINE" of folder "registry" of folder "drivers" of folder "HKEY_LOCAL_MACHINE" of folder "drivers" of folder "registry" of folder "drivers" of folder "registry" of folder "drivers"


In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True, #The model is first created on the Meta device (with empty weights) and the state dict is then loaded inside it (shard by shard in the case of a sharded checkpoint)
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, output_dir)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model.push_to_hub(new_model, max_shard_size='2GB')
tokenizer.push_to_hub(new_model)

pytorch_model-00003-of-00007.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00005-of-00007.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

pytorch_model-00007-of-00007.bin:   0%|          | 0.00/1.66G [00:00<?, ?B/s]

pytorch_model-00004-of-00007.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

pytorch_model-00006-of-00007.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

pytorch_model-00002-of-00007.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rishilearn/llama-2-7b-guanaco-dolly-mini/commit/95ae9d92d3ed68870cdf79f1e4c8e5eddb07a315', commit_message='Upload tokenizer', commit_description='', oid='95ae9d92d3ed68870cdf79f1e4c8e5eddb07a315', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# trainer.save_model(path)
# model = BertModel.from_pretrained("./test/saved_model/")
# for the future use of the model
#save_path = "path/to/save/model"  # Specify the path where you want to save the model
# model.save_pretrained(save_path)
# tokenizer.save_pretrained(save_path)
